In [2]:
from collections.abc import Iterable
import copy
import numpy as np
import scipy

In [7]:
class Vector(Iterable):
    def __init__(self, elems=None):
        if elems is None:
            elems = []
        self.__elems = elems

    def __getitem__(self, key):
        return self.__elems[key]

    def __setitem__(self, key, val):
        self.__elems[key] = val

    def __len__(self):
        return len(self.__elems)

    def __iter__(self):
        return self.__elems.__iter__()

    def __add__(self, other):
        if len(self) != len(other):
            raise ValueError("Vector must be of equal length")
        return Vector([a + b for a, b in zip(self, other)])

    def __neg__(self):
        return Vector([-a for a in self.__elems])

    def __sub__(self, other):
        if len(self) != len(other):
            raise ValueError("Vector must be of equal length")
        return Vector([a - b for a, b in zip(self, other)])

    def __mul__(self, other):
        if len(self) != len(other):
            raise ValueError("Vector must be of equal length")
        return Vector([a * b for a, b in zip(self, other)])

    def __rmul__(self, other):
        return Vector([other * a for a in self.__elems])

    def __str__(self):
        comma_list = ''.join([str(a) + ', ' for a in self.__elems])
        return '<' + comma_list[:-2] + '>'

    def append(self, val):
        self.__elems.append(val)


class Matrix:
    def __init__(self):
        self.__rows = []

    @staticmethod
    def create_from_list(list_of_lists):
        m = Matrix()
        for row in list_of_lists:
            m.add_row(Vector(row))
        return m

    @staticmethod
    def identity_matrix(n):
        I = [[1 if i == j  else 0 for i in range(n)] for j in range(n)]
        return Matrix.create_from_list(I)

    def number_of_rows(self):
        return len(self.__rows)

    def number_of_columns(self):
        return 0 if len(self.__rows) == 0 else len(self.__rows[0])

    def is_empty(self):
        return len(self.__rows) == 0

    def __len__(self):
        return self.number_of_rows() * self.number_of_columns()

    def __getitem__(self, item):
        return self.__rows[item]

    def __setitem__(self, key, new_row):
        self.__rows[key] = new_row

    def __str__(self):
        return ''.join([str(row) + '\n' for row in self.__rows])

    def add_row(self, new_row):
        if len(new_row) != self.number_of_columns() and not self.is_empty():
            raise ValueError("Matrix rows must be of equal length")
        self.__rows.append(new_row)

    def add_column(self, new_column):
        if len(new_column) != self.number_of_rows():
            raise ValueError("Matrix columns must be of equal length")
        for i in range(self.number_of_rows()):
            self.__rows[i].append(new_column[i])

    def subtract_rows(self, row1, row2):
        self.__rows[row1] -= self.__rows[row2]

    def multiply_row(self, key, scalar):
        self.__rows[key] *= scalar

    def swap_rows(self, row1, row2):
        self.__rows[row1], self.__rows[row2] = self.__rows[row2], self.__rows[row1]

    def multiply_column(self, column, scalar):
        for row in self.__rows:
            row[column] *= scalar

    def subtract_column(self, index1, index2):
        for row in self.__rows:
            row[index1] -= row[index2]

    def upper_triangular_matrix(self):
        matrix = copy.deepcopy(self)
        if matrix.number_of_rows() == matrix.number_of_columns():
            for j in range(matrix.number_of_columns()):
                find_nullifying_row = False
                for i in range(j, matrix.number_of_rows()):
                    if matrix[i][j] != 0 and find_nullifying_row == False:
                        matrix.swap_rows(i, j)
                        find_nullifying_row = True
                        continue
                    if find_nullifying_row:
                        a = matrix[i][j] / matrix[j][j]
                        matrix[i] = matrix[i] - a * matrix[j]
                if not find_nullifying_row:
                    raise ValueError("Macierz osobliwa")
        return matrix

    def det(self):
        matrix = self.upper_triangular_matrix()
        det = 1
        for i in range(len(self.__rows)):
            det *= matrix[i][i]
        return det
    
    def multiply_matrix_by_vector(self, vector: Vector) -> Vector:
        if self.number_of_columns() != len(vector):
            raise ValueError("Vector length must be of equal to number of columns in matrix")
        new_vector = Vector()
        for row in self.__rows:
            new_vector.append(sum(row * vector))
        return new_vector

In [4]:
def calculate_upper_triangular_matrix(A: Matrix, b: Vector):
    A, b = copy.deepcopy(A), copy.deepcopy(b)
    n = A.number_of_rows()
    for column in range(n):
        find_nullifying_row = False
        for row in range(column, n):
            if A[row][column] != 0 and find_nullifying_row == False:
                A.swap_rows(row, column)
                b[row], b[column] = b[column], b[row]
                find_nullifying_row = True
                continue
            if find_nullifying_row:
                multiplier = row_nullify_multiplier_for_column(
                    row_to_nullify=A[row],
                    base_row=A[column],
                    column=column
                )
                A[row] = A[row] - multiplier * A[column]
                b[row] = b[row] - multiplier * b[column]
        if not find_nullifying_row:
            raise ValueError("Macierz osobliwa")
    return A, b


def row_nullify_multiplier_for_column(row_to_nullify, base_row, column):
    return row_to_nullify[column] / base_row[column]


def gaussian_solver(A: Matrix, b: Vector) -> Vector:
    A, b = calculate_upper_triangular_matrix(A, b)
    solutions = Vector([0 for _ in range(A.number_of_rows())])
    for column in range(A.number_of_columns()-1, -1, -1):
        for row in range(column-1, -1, -1):
            multiplier = row_nullify_multiplier_for_column(
                row_to_nullify=A[row],
                base_row=A[column],
                column=column
            )
            A[row] = A[row] - multiplier * A[column]
            b[row] = b[row] - multiplier * b[column]
        solutions[column] = b[column] / A[column][column]
    return solutions

### Zadanie 4

In [5]:
A = Matrix.create_from_list(
    [[0, 0, 2, 1, 2],
    [0, 1, 0, 2, -1],
    [1, 2, 0, -2, 0],
    [0, 0, 0, -1, 1],
    [0, 1, -1, 1, -1]])
b = Vector([1, 1, -4, -2, -1])

print(gaussian_solver(A, b))
print(scipy.linalg.solve(A, b))

<2.0, -2.0, 1.0, 1.0, -1.0>
[ 2. -2.  1.  1. -1.]


### Zadanie 5

In [6]:
def polynomial_coeffs_lst(x0):
    return [x0 ** i for i in range(5)]


points = [(0, -1), (1, 1), (3, 3), (5, 2), (6, -2)]
A = []
b = []
for x, y in points:
    A.append(polynomial_coeffs_lst(x))
    b.append(y)

A = Matrix.create_from_list(A)
b = Vector(b)

print(gaussian_solver(A, b))
print(scipy.linalg.solve(A, b))

<-1.0, 2.6833333333333336, -0.8750000000000003, 0.21666666666666679, -0.02500000000000001>
[-1.          2.68333333 -0.875       0.21666667 -0.025     ]


### Zadanie 6

In [8]:
A = Matrix.create_from_list(
    [[3.50, 2.77, -0.76, 1.80],
     [-1.80, 2.68, 3.44, -0.09],
     [0.27, 5.07, 6.90, 1.61],
     [1.71, 5.45, 2.68, 1.71]])

b = Vector([7.31, 4.23, 13.85, 11.55])

x = gaussian_solver(A, b)
print(A.multiply_matrix_by_vector(x))
print(b)

<7.310000000000077, 4.2299999999999605, 13.850000000000003, 11.550000000000038>
<7.31, 4.23, 13.85, 11.55>


### Zadanie 7

In [12]:
A = Matrix.create_from_list(
    [[10, -2, -1, 2, 3, 1, -4, 7],
     [5, 11, 3, 10, -3, 3, 3, -4],
     [7, 12, 1, 5, 3, -12, 2, 3],
     [8, 7, -2, 1, 3, 2, 2, 4],
     [2, -15, -1, 1, 4, -1, 8, 3],
     [4, 2, 9, 1, 12, -1, 4, 1],
     [-1, 4, -7, -1, 1, 1, -1, -3],
     [-1, 3, 4, 1, 3, -4, 7, 6]])

b = Vector([0, 12, -5, 3, -25, -26, 9, -7])

print(gaussian_solver(A, b))
print(scipy.linalg.solve(A, b))
print(np.linalg.solve(A, b))

<-0.9999999999999982, 0.9999999999999997, -1.0000000000000018, 0.9999999999999991, -1.000000000000001, 0.9999999999999993, -0.9999999999999991, 0.9999999999999987>
[-1.  1. -1.  1. -1.  1. -1.  1.]
[-1.  1. -1.  1. -1.  1. -1.  1.]


### Zadanie 8

### Zadanie 9